#**Resume Scanner Application**
- Upload Resume.pdf from Gradio UI
- Get Valid JSON Output like,
  - Name
  - Email
  - Phone Number
  - Skills
  - Experience

##**0. Install Dependencies**

In [1]:
!pip install pydantic openai gradio pdfplumber

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.8/42.8 kB 1.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.2/48.2 kB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.1/54.1 MB 15.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 322.9/322.9 kB 15.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.2/60.2 kB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 72.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.2/95.2 kB 6.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.9/2.9 MB 59.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.5/11.5 MB 68.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.0/72.0 kB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.5/62.5 kB 4.9 MB/s eta 0:00:00


##**1. Import Required Modules**

In [2]:
import gradio as gr
import pdfplumber
import json
from openai import OpenAI
from pydantic import BaseModel, ValidationError
import re

##**2. Get API key from Secret**

In [3]:
from google.colab import userdata
# Retrieve the API key from Colab's secrets
api_key = userdata.get('openai_api_key')

##**3. Set Up OpenAI Client**

In [4]:
# Set your OpenAI API key
client = OpenAI(api_key=api_key)

##**4. Define Pydantic Models**

In [5]:
# --- Step 1: Pydantic model ---
class ResumeData(BaseModel):
    name: str
    email: str
    phone: str
    skills: list[str]
    experience: str

##**5. Extract text from PDF**

In [6]:
# --- Step 2: Extract text from PDF ---
def extract_text_from_pdf(pdf_file):
    with pdfplumber.open(pdf_file.name) as pdf:
        text = "\n".join([page.extract_text() for page in pdf.pages if page.extract_text()])
    return text

##**6. Parse the Resume**

In [7]:
def parse_resume(pdf_file):
    text = extract_text_from_pdf(pdf_file)

    prompt = f"""
    Extract the following details from the resume text below:
    - name
    - email
    - phone
    - skills (as a list)
    - experience (a brief summary)

    Respond only in JSON format:
    Resume Text:
    \"\"\"{text}\"\"\"
    """

    try:
        response = client.chat.completions.create(
            model="gpt-4o-mini",
            messages=[{"role": "user", "content": prompt}]
        )

        raw_output = response.choices[0].message.content.strip()

        # Extract JSON only
        match = re.search(r"\{.*\}", raw_output, re.DOTALL)
        if match:
            json_text = match.group(0)
            data = json.loads(json_text)
            validated = ResumeData.model_validate(data)
            return validated.model_dump_json(indent=2)
        else:
            return "❌ Could not find JSON in response."

    except (ValidationError, json.JSONDecodeError) as e:
        return f"❌ Validation or parsing error:\n{str(e)}"
    except Exception as e:
        return f"❌ API error:\n{str(e)}"

##**7. Gradio Interface**

In [8]:
app = gr.Interface(
    fn=parse_resume,
    inputs=gr.File(label="Upload Resume PDF", file_types=[".pdf"]),
    outputs=gr.Code(label="Structured Resume JSON", language="json"),
    title="🧠 Resume Parser with OpenAI + Gradio",
    description="Upload a resume PDF. The app extracts structured info (name, email, skills, etc.) using GPT."
)

app.launch()

It looks like you are running Gradio on a hosted a Jupyter notebook. For the Gradio app to work, sharing must be enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://46008763babac5c2d4.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
